In [11]:

from random import randint
from PIL import Image
import numpy as np
import pickle
import pycuda.driver as cuda
import pycuda.autoinit
import uff
import tensorrt as trt
import ctypes

from matplotlib import pyplot as plt
from six.moves import cPickle

import logging
logger = logging.getLogger()
logger.setLevel(logging.ERROR)
logging.debug("test")

In [12]:
## information about data needed to parse model and build engine
class ModelData(object):
    MODEL_FILE = "/home/vtpc/Documents/Alvils/tensorrt/libs/caffe/models/bvlc_googlenet/weights.caffemodel"
    DEPLOY_FILE = "/home/vtpc/Documents/Alvils/tensorrt/libs/caffe/models/bvlc_googlenet/deploy.prototxt"
    INPUT_SHAPE = (3, 227, 227) # always nchw
    OUTPUT_NAME = "prob"# 227 dim: 227
    DTYPE = trt.float32

In [13]:
TRT_LOGGER = trt.Logger(trt.Logger.ERROR)
# parses model to trt
with trt.Builder(TRT_LOGGER) as builder, builder.create_network() as network, trt.CaffeParser() as parser:
    # builder argums for optimization
    builder.max_batch_size = 1
    builder.max_workspace_size = 1 << 30    

    # Load the Caffe model and parse it in order to populate the TensorRT network.
    # This function returns an object that we can query to find tensors by name.
    model_tensors = parser.parse(deploy=ModelData.DEPLOY_FILE, model=ModelData.MODEL_FILE, network=network, dtype=ModelData.DTYPE)
    # For Caffe, we need to manually mark the output of the network.
    # Since we know the name of the output tensor, we can find it in model_tensors.
    network.mark_output(model_tensors.find(ModelData.OUTPUT_NAME))

    # builds engine
    with builder.build_cuda_engine(network) as engine:
        with open("fp32.engine", "wb") as f:
            f.write(engine.serialize())
